In [3]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [4]:
class ProductNormalizer:
    def __init__(self):
        self.patterns = {
            'weight': r'(\d+[,.]?\d*)\s*(г|кг|гр|g|kg)',
            'volume': r'(\d+[,.]?\d*)\s*(л|мл|L|ml)',
            'dimensions': r'(\d+[хx]\d+[хx]\d+мм?)',
            'count': r'(\d+)\s*(шт|уп|пач|таб|предм)',
            'percentage': r'(\d+[,.]?\d*)%',
            'diameter': r'[dD]\s*[=]?\s*(\d+[,.]?\d*)\s*(м|см|мм)',
            'articul': r'[A-Z]{2,}\d+[-_]?\d*|[A-Z]{1,2}-\d+',
            'model': r'[A-Z]{1,5}\d+[-]?[A-Z]?\d*',
        }
        
        # Словари для извлечения
        self.brands = {
            'sony', 'samsung', 'epson', 'nivea', 'gillette', 'purina', 'vishi', 
            'snickers', 'twix', 'milky way', 'm&m', 'bounty', 'kellogg', 'fitness',
            'nestle', 'alpengold', 'lukoil', 'grandex', 'egger', 'magicstar', 'grandcaratt',
            'turtlewax', 'fest', 'energon', 'coffeeone', 'actimel', 'activia', 'maggi',
            'borjomi', 'toughpix', 'abat', 'cricket', 'flash', 'viola', 'lorenz',
            'coppini', 'always', 'greenfield', 'durex', 'frapin', 'glencadam'
        }
        
        self.materials = {
            'дерево', 'металл', 'пластик', 'стекло', 'керамика', 'сталь', 'алюминий',
            'картон', 'бумага', 'текстиль', 'кожа', 'резина', 'силикон', 'акрил',
            'гранит', 'мрамор', 'полипропилен', 'полистирол', 'полиуретан'
        }
        
        self.colors = {
            'черный', 'белый', 'красный', 'синий', 'зеленый', 'желтый', 'оранжевый',
            'фиолетовый', 'розовый', 'коричневый', 'серый', 'серебристый', 'золотой',
            'прозрачный', 'матовый', 'глянцевый'
        }

    def extract_structured_attributes(self, text):
        """Извлечение структурированных атрибутов с помощью RegEx"""
        attributes = {}
        
        for attr_type, pattern in self.patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Берем первое совпадение
                if attr_type in ['weight', 'volume', 'percentage', 'diameter']:
                    attributes[attr_type] = matches[0][0] + matches[0][1]
                elif attr_type == 'count':
                    attributes[attr_type] = matches[0][0] + ' ' + matches[0][1]
                elif attr_type == 'dimensions':
                    attributes[attr_type] = matches[0]
                elif attr_type in ['articul', 'model']:
                    attributes[attr_type] = matches[0]
        
        return attributes

    def extract_brand(self, text):
        """Извлечение бренда из текста"""
        text_lower = text.lower()
        for brand in self.brands:
            if brand in text_lower:
                return brand.title()
        return None

    def extract_materials(self, text):
        """Извлечение материалов"""
        text_lower = text.lower()
        found_materials = []
        for material in self.materials:
            if material in text_lower:
                found_materials.append(material)
        return found_materials if found_materials else None

    def extract_colors(self, text):
        """Извлечение цветов"""
        text_lower = text.lower()
        found_colors = []
        for color in self.colors:
            if color in text_lower:
                found_colors.append(color)
        return found_colors if found_colors else None

    def extract_product_type(self, text, class_name):
        """Извлечение типа продукта"""
        # Убираем бренды и сосредотачиваемся на основном типе
        text_clean = text.lower()
        for brand in self.brands:
            text_clean = text_clean.replace(brand, '')
        
        # Ищем ключевые слова, характеризующие тип
        type_keywords = [
            'набор', 'комплект', 'пакет', 'упаковка', 'коробка', 'бутылка',
            'банка', 'пачка', 'флакон', 'тюбик', 'баночка', 'мешок'
        ]
        
        for keyword in type_keywords:
            if keyword in text_clean:
                return keyword
        
        return class_name.split()[-1] if class_name else 'товар'

    def extract_all_attributes(self, text, class_name):
        """Основной метод извлечения всех атрибутов"""
        if pd.isna(text) or text == '':
            return {}
            
        attributes = self.extract_structured_attributes(text)
        attributes['brand'] = self.extract_brand(text)
        attributes['materials'] = self.extract_materials(text)
        attributes['colors'] = self.extract_colors(text)
        attributes['product_type'] = self.extract_product_type(text, class_name)
        
        # Очистка от None значений
        return {k: v for k, v in attributes.items() if v is not None and v != ''}

    def create_short_name(self, attributes, class_name):
        """Создание нормализованного краткого наименования"""
        parts = []
        
        # Бренд
        if attributes.get('brand'):
            parts.append(attributes['brand'])
        
        # Тип продукта
        parts.append(attributes.get('product_type', class_name))
        
        # Модель/артикул
        if attributes.get('model'):
            parts.append(attributes['model'])
        elif attributes.get('articul'):
            parts.append(attributes['articul'])
        
        # Основная характеристика (вес/объем/размеры)
        if attributes.get('weight'):
            parts.append(attributes['weight'])
        elif attributes.get('volume'):
            parts.append(attributes['volume'])
        elif attributes.get('dimensions'):
            parts.append('x'.join(re.findall(r'\d+', attributes['dimensions'][:3])) + 'мм')
        
        return ' '.join(parts)

    def create_full_name(self, attributes, class_name):
        """Создание нормализованного полного наименования"""
        parts = []
        
        # Бренд
        if attributes.get('brand'):
            parts.append(attributes['brand'])
        
        # Тип продукта
        product_type = attributes.get('product_type', class_name)
        parts.append(product_type)
        
        # Детальные характеристики
        specs = []
        
        # Модель/артикул
        if attributes.get('model'):
            specs.append(f"модель {attributes['model']}")
        if attributes.get('articul'):
            specs.append(f"арт. {attributes['articul']}")
        
        # Размерные характеристики
        if attributes.get('weight'):
            specs.append(f"вес {attributes['weight']}")
        if attributes.get('volume'):
            specs.append(f"объем {attributes['volume']}")
        if attributes.get('dimensions'):
            dims = 'x'.join(re.findall(r'\d+', attributes['dimensions'][:3]))
            specs.append(f"размер {dims}мм")
        if attributes.get('count'):
            specs.append(attributes['count'])
        
        # Материалы и цвета
        if attributes.get('materials'):
            specs.append(f"материал: {', '.join(attributes['materials'])}")
        if attributes.get('colors'):
            specs.append(f"цвет: {', '.join(attributes['colors'])}")
        
        # Собираем полное название
        if specs:
            parts.append('(' + ', '.join(specs) + ')')
        
        return ' '.join(parts)

In [18]:
def normalize_dataframe(df):
    """
    Основная функция для нормализации датафрейма
    """
    normalizer = ProductNormalizer()
    
    results = []
    
    for idx, row in df.iterrows():
        try:
            class_code = row['Hierarchy_MTR_Class']
            class_name = row['Hierarchy_MTR_Name']
            short_name = row['SHORT_NAME/ru_RU']
            full_name = row['FULL_NAME/ru_RU']
            cscd_id = row['CSCD_ID']
            
            # Проверяем на NaN значения
            if pd.isna(full_name) or full_name == '':
                attributes = {}
            else:
                # Извлекаем атрибуты из полного наименования
                attributes = normalizer.extract_all_attributes(str(full_name), str(class_name))
            
            # Создаем нормализованные названия
            normalized_short = normalizer.create_short_name(attributes, str(class_name))
            normalized_full = normalizer.create_full_name(attributes, str(class_name))
            
            results.append({
                'Hierarchy_MTR_Class': class_code,
                'Hierarchy_MTR_Name': class_name,
                'CSCD_ID': cscd_id,
                'Original_Short_Name': short_name,
                'Original_Full_Name': full_name,
                'Normalized_Short_Name': normalized_short,
                'Normalized_Full_Name': normalized_full,
                'Extracted_Attributes': str(attributes)
            })
            
        except Exception as e:
            print(f"Ошибка при обработке строки {idx}: {e}")
            print(f"Данные строки: {row.to_dict()}")
            # Добавляем запись с пустыми атрибутами в случае ошибки
            results.append({
                'Hierarchy_MTR_Class': row.get('Hierarchy_MTR_Class', ''),
                'Hierarchy_MTR_Name': row.get('Hierarchy_MTR_Name', ''),
                'CSCD_ID': row.get('CSCD_ID', ''),
                'Original_Short_Name': row.get('SHORT_NAME/ru_RU', ''),
                'Original_Full_Name': row.get('FULL_NAME/ru_RU', ''),
                'Normalized_Short_Name': '',
                'Normalized_Full_Name': '',
                'Extracted_Attributes': '{}'
            })
    
    return pd.DataFrame(results)

In [19]:
# ОБНОВЛЕННАЯ ФУНКЦИЯ ДЛЯ ИЗВЛЕЧЕНИЯ СТРУКТУРИРОВАННЫХ АТРИБУТОВ
def extract_structured_attributes_fixed(self, text):
    """Извлечение структурированных атрибутов с помощью RegEx (исправленная версия)"""
    attributes = {}
    
    for attr_type, pattern in self.patterns.items():
        try:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Берем первое совпадение и безопасно обрабатываем
                match = matches[0]
                
                if isinstance(match, tuple):
                    if len(match) >= 2:
                        if attr_type in ['weight', 'volume', 'percentage', 'diameter']:
                            attributes[attr_type] = str(match[0]) + str(match[1])
                        elif attr_type == 'count':
                            attributes[attr_type] = str(match[0]) + ' ' + str(match[1])
                        elif attr_type == 'dimensions':
                            attributes[attr_type] = str(match[0])
                    else:
                        attributes[attr_type] = str(match[0])
                else:
                    attributes[attr_type] = str(match)
                    
        except Exception as e:
            # Пропускаем атрибут в случае ошибки
            continue
    
    return attributes

# ЗАМЕНЯЕМ МЕТОД В КЛАССЕ
ProductNormalizer.extract_structured_attributes = extract_structured_attributes_fixed

In [20]:
# Функции для анализа результатов
def analyze_results(normalized_df):
    """Анализ качества нормализации"""
    print("=== АНАЛИЗ РЕЗУЛЬТАТОВ НОРМАЛИЗАЦИИ ===\n")
    
    total_rows = len(normalized_df)
    print(f"Всего обработано записей: {total_rows}")
    
    # Статистика по извлеченным атрибутам
    attributes_stats = {
        'brand': 0,
        'weight': 0,
        'volume': 0,
        'dimensions': 0,
        'count': 0,
        'materials': 0,
        'colors': 0
    }
    
    for idx, row in normalized_df.iterrows():
        attrs = eval(row['Extracted_Attributes'])
        for attr in attributes_stats.keys():
            if attr in attrs:
                attributes_stats[attr] += 1
    
    print("\n=== СТАТИСТИКА ИЗВЛЕЧЕННЫХ АТРИБУТОВ ===")
    for attr, count in attributes_stats.items():
        percentage = (count / total_rows) * 100
        print(f"{attr}: {count} ({percentage:.1f}%)")
    
    # Показать несколько примеров
    print("\n=== ПРИМЕРЫ НОРМАЛИЗАЦИИ ===")
    for i in range(min(5, len(normalized_df))):
        print(f"\nПример {i+1}:")
        print(f"Оригинальное краткое: {normalized_df.iloc[i]['Original_Short_Name']}")
        print(f"Нормализованное краткое: {normalized_df.iloc[i]['Normalized_Short_Name']}")
        print(f"Оригинальное полное: {normalized_df.iloc[i]['Original_Full_Name'][:100]}...")
        print(f"Нормализованное полное: {normalized_df.iloc[i]['Normalized_Full_Name']}")
        print(f"Извлеченные атрибуты: {normalized_df.iloc[i]['Extracted_Attributes']}")
        print("-" * 80)

In [ ]:
test_data = {
    'Hierarchy_MTR_Class': ['701090900000000', '702080500000000', '701090600000000'],
    'Hierarchy_MTR_Name': ['Мебель и предметы интерьера', 'Конфеты', 'Бытовая техника'],
    'SHORT_NAME/ru_RU': [
        'Кофе-корнер тип 1 2650х700х900мм',
        'Набор M&M\'s & Friends Посылка 452г', 
        'Проектор Epson EB-L200F V11H990040 кмп'
    ],
    'FULL_NAME/ru_RU': [
        'Кофе-корнер тип 1 2650х700х900мм,столешница акрил кам.Grandex C-802 Piacenza,корп.и дверц.Egger H1318 ST10 дикий дуб...',
        'Набор M&M\'s & Friends Посылка подарочный: шоколадные батончики Snickers, Snickers minis... 452г',
        'Проектор Epson EB-L200F V11H990040 в комплекте с универсальным потолочным креплением Wize Pro PR3XL-W...'
    ],
    'CSCD_ID': ['12345', '67890', '54321']  # Добавляем тестовые номера номенклатуры
}

df = pd.DataFrame(test_data)

In [15]:
# ЗАГРУЗКА ДАННЫХ ИЗ EXCEL ФАЙЛА
df = pd.read_excel('потребительские.xlsx', engine='openpyxl')

In [21]:
# ЯЧЕЙКА ДЛЯ ЗАПУСКА В JUPYTER NOTEBOOK
print("Загрузка и нормализация данных...")




# НОРМАЛИЗАЦИЯ ДАННЫХ
normalized_df = normalize_dataframe(df)

# Функция для сохранения атрибутов в развернутом виде в Excel
def save_attributes_to_excel(normalized_df, filename='attributes_expanded.xlsx'):
    """Сохраняет атрибуты в развернутом формате: класс, имя класса, код номенклатуры, атрибут, значение"""
    
    expanded_data = []
    
    for idx, row in normalized_df.iterrows():
        class_code = row['Hierarchy_MTR_Class']
        class_name = row['Hierarchy_MTR_Name']
        cscd_id = row['CSCD_ID']
        attributes = eval(row['Extracted_Attributes'])
        
        # Добавляем каждый атрибут как отдельную строку
        for attr_name, attr_value in attributes.items():
            # Обрабатываем списки (материалы, цвета)
            if isinstance(attr_value, list):
                for item in attr_value:
                    expanded_data.append({
                        'Class_Code': class_code,
                        'Class_Name': class_name,
                        'CSCD_ID': cscd_id,
                        'Attribute': attr_name,
                        'Attribute_Value': item
                    })
            else:
                expanded_data.append({
                    'Class_Code': class_code,
                    'Class_Name': class_name,
                    'CSCD_ID': cscd_id,
                    'Attribute': attr_name,
                    'Attribute_Value': attr_value
                })
    
    # Создаем DataFrame
    expanded_df = pd.DataFrame(expanded_data)
    
    # Сохраняем в Excel
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        expanded_df.to_excel(writer, sheet_name='Attributes', index=False)
    
    return expanded_df

# Сохраняем атрибуты в развернутом виде в Excel
expanded_attributes_df = save_attributes_to_excel(normalized_df, 'attributes_expanded.xlsx')

# АНАЛИЗ РЕЗУЛЬТАТОВ
analyze_results(normalized_df)

# СОХРАНЕНИЕ РЕЗУЛЬТАТОВ В EXCEL С НЕСКОЛЬКИМИ ЛИСТАМИ
with pd.ExcelWriter('normalized_products_results.xlsx', engine='openpyxl') as writer:
    # Лист с нормализованными данными
    normalized_df.to_excel(writer, sheet_name='Normalized_Products', index=False)
    
    # Лист с развернутыми атрибутами
    expanded_attributes_df.to_excel(writer, sheet_name='Expanded_Attributes', index=False)
    
    # Лист со статистикой
    stats_data = {
        'Metric': [
            'Всего обработано записей',
            'Уникальных классов',
            'Уникальных категорий', 
            'Всего извлеченных атрибутов',
            'Уникальных типов атрибутов'
        ],
        'Value': [
            len(normalized_df),
            normalized_df['Hierarchy_MTR_Class'].nunique(),
            normalized_df['Hierarchy_MTR_Name'].nunique(),
            len(expanded_attributes_df),
            expanded_attributes_df['Attribute'].nunique()
        ]
    }
    stats_df = pd.DataFrame(stats_data)
    stats_df.to_excel(writer, sheet_name='Statistics', index=False)

print(f"\nРезультаты сохранены в файл: normalized_products_results.xlsx")
print("Файл содержит листы:")
print("- Normalized_Products: нормализованные наименования")
print("- Expanded_Attributes: развернутые атрибуты")
print("- Statistics: статистика по обработке")

# ДОПОЛНИТЕЛЬНАЯ ВИЗУАЛИЗАЦИЯ
print("\n=== ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ ===")
print(f"Уникальных классов: {normalized_df['Hierarchy_MTR_Class'].nunique()}")
print(f"Уникальных категорий: {normalized_df['Hierarchy_MTR_Name'].nunique()}")
print(f"Всего извлеченных атрибутов: {len(expanded_attributes_df)}")
print(f"Уникальных типов атрибутов: {expanded_attributes_df['Attribute'].nunique()}")

# Показать пример развернутых атрибутов
print("\n=== ПРИМЕР РАЗВЕРНУТЫХ АТРИБУТОВ ===")
display(expanded_attributes_df.head(10))

# Показать все результаты нормализации
print("\n=== ПОЛНЫЕ РЕЗУЛЬТАТЫ НОРМАЛИЗАЦИИ ===")
pd.set_option('display.max_colwidth', 100)
display(normalized_df[['CSCD_ID', 'Original_Short_Name', 'Normalized_Short_Name', 'Normalized_Full_Name']])

Загрузка и нормализация данных...
=== АНАЛИЗ РЕЗУЛЬТАТОВ НОРМАЛИЗАЦИИ ===

Всего обработано записей: 219556

=== СТАТИСТИКА ИЗВЛЕЧЕННЫХ АТРИБУТОВ ===
brand: 2101 (1.0%)
weight: 64634 (29.4%)
volume: 41169 (18.8%)
dimensions: 1270 (0.6%)
count: 11954 (5.4%)
materials: 15065 (6.9%)
colors: 12939 (5.9%)

=== ПРИМЕРЫ НОРМАЛИЗАЦИИ ===

Пример 1:
Оригинальное краткое: Кофе-корнер тип 1 2650х700х900мм
Нормализованное краткое: Egger интерьера H1318 265мм
Оригинальное полное: Кофе-корнер тип 1 2650х700х900мм,столешница акрил кам.Grandex C-802 Piacenza,корп.и дверц.Egger H131...
Нормализованное полное: Egger интерьера (модель H1318, арт. C-802, размер 265мм, материал: акрил)
Извлеченные атрибуты: {'dimensions': '2650х700х900мм', 'articul': 'C-802', 'model': 'H1318', 'brand': 'Egger', 'materials': ['акрил'], 'product_type': 'интерьера'}
--------------------------------------------------------------------------------

Пример 2:
Оригинальное краткое: Кофе-корнер тип 2 1500х700х900мм
Нормализованное

,Class_Code,Class_Name,CSCD_ID,Attribute,Attribute_Value
0,701090900000000,Мебель и предметы интерьера,2378964,dimensions,2650х700х900мм
1,701090900000000,Мебель и предметы интерьера,2378964,articul,C-802
2,701090900000000,Мебель и предметы интерьера,2378964,model,H1318
3,701090900000000,Мебель и предметы интерьера,2378964,brand,Egger
4,701090900000000,Мебель и предметы интерьера,2378964,materials,акрил
5,701090900000000,Мебель и предметы интерьера,2378964,product_type,интерьера
6,701090900000000,Мебель и предметы интерьера,2378975,dimensions,1500х700х900мм
7,701090900000000,Мебель и предметы интерьера,2378975,articul,C-802
8,701090900000000,Мебель и предметы интерьера,2378975,model,H1318
9,701090900000000,Мебель и предметы интерьера,2378975,brand,Egger



=== ПОЛНЫЕ РЕЗУЛЬТАТЫ НОРМАЛИЗАЦИИ ===


,CSCD_ID,Original_Short_Name,Normalized_Short_Name,Normalized_Full_Name
0,2378964,Кофе-корнер тип 1 2650х700х900мм,Egger интерьера H1318 265мм,"Egger интерьера (модель H1318, арт. C-802, размер 265мм, материал: акрил)"
1,2378975,Кофе-корнер тип 2 1500х700х900мм,Egger интерьера H1318 150мм,"Egger интерьера (модель H1318, арт. C-802, размер 150мм, материал: акрил)"
2,2379048,Стол барный тип 1 2000х1000х1100мм,Egger интерьера H1318 200мм,"Egger интерьера (модель H1318, арт. ST10, размер 200мм, материал: металл, цвет: матовый)"
3,2379051,Стол барный тип 2 2500х1000х1100мм,Egger интерьера H1318 250мм,"Egger интерьера (модель H1318, арт. ST10, размер 250мм, материал: металл, цвет: матовый)"
4,2379055,Стол барный тип 3 2500х650х1100мм,Egger интерьера H1318 250мм,"Egger интерьера (модель H1318, арт. ST10, размер 250мм, материал: металл, цвет: матовый)"
...,...,...,...,...
219551,3121148,Пуф,интерьера,интерьера
219552,1311463,Мяч,товары,товары
219553,2246547,Бра,интерьера,интерьера
219554,3540396,Пуф,интерьера,интерьера
